### Libraries and datasets required

original dataset from  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \

In [130]:
#!pip install tfrecorder
#!pip install apache-beam[gcp]

In [81]:
import os
import pandas as pd
import tfrecorder

### GCS Fuse to be able to use os utilities on GCS without copying data

In [ ]:
!gcsfuse --implicit-dirs mchrestkha-demo-env-ml-examples /home/jupyter/gcs/ 
#!fusermount -u /home/jupyter/gcs/

### Collect all image URIs

In [132]:
myDir='/home/jupyter/gcs/catsdogs/'
format='.jpg'
fileList = []
for root, dirs, files in os.walk(myDir, topdown=False):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
                
fileList[:10]

['/home/jupyter/gcs/catsdogs/train/cats/cat.0.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.1.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.10.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.100.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.101.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.102.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.103.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.104.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.105.jpg',
 '/home/jupyter/gcs/catsdogs/train/cats/cat.106.jpg']

In [107]:
df=pd.DataFrame(fileList)
df.columns = ['image_uri']
df.head()

,image_uri
0,/home/jupyter/gcs/catsdogs/train/cats/cat.0.jpg
1,/home/jupyter/gcs/catsdogs/train/cats/cat.1.jpg
2,/home/jupyter/gcs/catsdogs/train/cats/cat.10.jpg
3,/home/jupyter/gcs/catsdogs/train/cats/cat.100.jpg
4,/home/jupyter/gcs/catsdogs/train/cats/cat.101.jpg


In [108]:
df.loc[df['image_uri'].str.contains('train'), 'split'] = 'TRAIN'
df.loc[df['image_uri'].str.contains('validation'), 'split'] = 'VALIDATION'
df.loc[df['image_uri'].str.contains('|'.join(['train/cats', 'validation/cats'])), 'label'] = 'cats'
df.loc[df['image_uri'].str.contains('|'.join(['train/dogs', 'validation/dogs'])), 'label'] = 'dogs'
df = df[['split', 'image_uri', 'label']]
df.head()

,split,image_uri,label
0,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.0.jpg,cats
1,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.1.jpg,cats
2,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.10.jpg,cats
3,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.100.jpg,cats
4,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.101.jpg,cats


In [112]:
dflocal=df.copy()
dfgcs=df.copy()
dfgcs['image_uri'] = dfgcs['image_uri'].str.replace('/home/jupyter/gcs','gs://mchrestkha-demo-env-ml-examples')

In [124]:
dflocal=dflocal.head(3)
dflocal.head()

,split,image_uri,label
0,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.0.jpg,cats
1,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.1.jpg,cats
2,TRAIN,/home/jupyter/gcs/catsdogs/train/cats/cat.10.jpg,cats


In [123]:
dfgcs=dfgcs.head(3)
dfgcs.head()

,split,image_uri,label
0,TRAIN,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,cats
1,TRAIN,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,cats
2,TRAIN,gs://mchrestkha-demo-env-ml-examples/catsdogs/...,cats


In [126]:
dflocal.to_csv('datalocal.csv', index=False)  
dflocal = pd.read_csv('datalocal.csv')

#dflocal.tensorflow.to_tfr(output_dir='/home/jupyter/tfrecords/')


In [133]:
#dfgcs.to_csv('datagcs.csv', index=False)  
#dfgcs = pd.read_csv('datagcs.csv')

#dfgcs.tensorflow.to_tfr(output_dir='gs://mchrestkha-demo-env-ml-examples/catsdogs/tfrecords')

dfgcs.tensorflow.to_tfr(
    output_dir='gs://mchrestkha-demo-env-ml-examples/catsdogs/tfrecords',
    runner='DataflowRunner',
    project='mchrestkha-demo-env',
    region='us-west1')

RuntimeError: The file /opt/conda/lib/python3.7/site-packages/tfrecorder/../setup.py cannot be found. It was specified in the --setup_file command line option.